required package installation

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge iris

Model and training

In [ ]:
# MODEL ARCHITECTURE TAKEN FROM  "https://github.com/lixiaolei1982/Keras-Implementation-of-U-Net-R2U-Net-Attention-U-Net-Attention-R2U-Net.-"
import iris
import iris.analysis.cartography
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np
import datetime

import tensorflow as tf
from tensorflow.keras.layers import multiply,add,Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, UpSampling2D, UpSampling3D, Add, BatchNormalization, Input, Activation, Lambda, Cropping2D,Concatenate,Dropout
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp

import glob

from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard

from tensorflow.keras.optimizers import Adam

tf.get_logger().setLevel('ERROR')
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)


def unet(img_w, img_h,channel, n_label=1, data_format='channels_last'):
    inputs = Input(( img_w, img_h,channel),name='inputs')
    x = inputs
    depth = 3
    features = 64
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format= data_format)(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        # attention_up_and_concate(x,[skips[i])
        x = UpSampling2D(size=(2, 2), data_format=data_format)(x)
        x = K.concatenate([skips[i], x], axis=3)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = Activation('linear',name='outputs')(conv6)
    model = Model(inputs=inputs, outputs=conv7)

    #model.compile(optimizer=Adam(lr=1e-5), loss=[focal_loss()], metrics=['accuracy', dice_coef])
    return model
########################################################################################################
#Attention U-Net
def att_unet(img_w, img_h,channel, n_label=1, data_format='channels_last'):
    inputs = Input(( img_w, img_h,channel),name='inputs')
    x = inputs
    depth = 3
    features = 64
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format=data_format)(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = Activation('linear',name='outputs')(conv6)
    model = Model(inputs=inputs, outputs=conv7)

    #model.compile(optimizer=Adam(lr=1e-5), loss=[focal_loss()], metrics=['accuracy', dice_coef])
    return model


def rec_res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

                  padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer

    layer = skip_layer
    for j in range(2):

        for i in range(2):
            if i == 0:

                layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                    layer)
                if batch_normalization:
                    layer1 = BatchNormalization()(layer1)
                layer1 = Activation('relu')(layer1)
            layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                add([layer1, layer]))
            if batch_normalization:
                layer1 = BatchNormalization()(layer1)
            layer1 = Activation('relu')(layer1)
        layer = layer1

    out_layer = add([layer, skip_layer])
    return out_layer


def attention_up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])
    return concate
def attention_block_2d(x, g, inter_channel, data_format='channels_first'):
    # theta_x(?,g_height,g_width,inter_channel)

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    # phi_g(?,g_height,g_width,inter_channel)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    # f(?,g_height,g_width,inter_channel)

    f = Activation('relu')(add([theta_x, phi_g]))

    # psi_f(?,g_height,g_width,1)

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    # rate(?,x_height,x_width)

    # att_x(?,x_height,x_width,x_channel)

    att_x = multiply([x, rate])

    return att_x
def att_r2_unet(img_w, img_h,channel=2, n_label=1, data_format='channels_last'):
    inputs = Input(( img_w, img_h,channel),name='inputs')
    x = inputs
    depth =3
    features = 64
    skips = []
    for i in range(depth):
        x = rec_res_block(x, features, data_format=data_format)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format=data_format)(x)

        features = features * 2

    x = rec_res_block(x, features, data_format=data_format)

    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = rec_res_block(x, features, data_format=data_format)

    conv6 = Conv2D(n_label, (1, 1), padding='same', data_format=data_format)(x)
    conv7 = Activation('linear',name='outputs')(conv6)
    model = Model(inputs=inputs, outputs=conv7)
    #model.compile(optimizer=Adam(lr=1e-6), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model

tf.get_logger().setLevel('ERROR')
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)




gpp_test_min=0
gpp_test_max=0

cubes_tas = []
def getData(dir,time_constraint_train,time_constraint_test):
  list1 = []


  #dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/annual/*CMIP6_BCC*.nc'

  

  for name in glob.glob(dir):
      list1.append(name)

  cubes_tas = iris.load(list1)

  # equalise_attributes(cubes_tas)
  # unify_time_units(cubes_tas)
  # names = [cube.metadata.name() for cube in cubes ]
  # unique_names = list(OrderedDict.fromkeys(names))


  for z in cubes_tas:
      try:

          if (z.coord('season_year')):
              year = z.coord('season_year')
              # Fix the names. Latitude values, units and
              year.var_name = 'year'
              year.long_name = 'year'
      except:
          print("An exception occurred aux")

  # Fix the names. Latitude values, units and


  for z in cubes_tas:
      try:
          z.remove_coord("clim_season")
      except:
          print("An exception occurred remove cilmate")

  for z in cubes_tas:
      try:
          if (z._names[0] == 'precipitation_flux'):
              z.convert_units('kg m-2 year-1')
          if (z._names[0] == 'air_temperature'):
              z.convert_units('celsius')
          if (z._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
              z.convert_units('kg m-2 year-1')

      except:
          print("An exception occurred conversion")

  #time_constraint_train = iris.Constraint(year=lambda cell: 1900 < cell < 2300)
  #time_constraint_test = iris.Constraint(year=lambda cell: 1860 < cell < 1900)

  tas_array_train = []
  pr_array_train = []
  gpp_array_train = []

  tas_array_test = []
  pr_array_test = []
  gpp_array_test = []
  mask_train = []
  mask_test=[]

  for z in cubes_tas:
      try:
          if (z._names[0] == 'precipitation_flux'):
              temp = z.extract(time_constraint_train)
              mask_train.append(temp.data._mask)
              pr_array_train.append(temp.data.filled(0))
              temp = z.extract(time_constraint_test)
              mask_test.append(temp.data._mask)             
              pr_array_test.append(temp.data.filled(0))             
          if (z._names[0] == 'air_temperature'):
              temp = z.extract(time_constraint_train)
              tas_array_train.append(temp.data.filled(0))
              temp = z.extract(time_constraint_test)
              tas_array_test.append(temp.data.filled(0))
          if (z._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
              temp = z.extract(time_constraint_train)
              gpp_array_train.append(temp.data.filled(0))
              temp = z.extract(time_constraint_test)
              gpp_array_test.append(temp.data.filled(0))

      except:
          print("An exception occurred  filling data")

  tas_array_train = np.concatenate(tas_array_train)
  pr_array_train = np.concatenate(pr_array_train)


  gpp_array_train = np.concatenate(gpp_array_train)

  gpp_array_train = np.subtract(gpp_array_train, gpp_array_train.min()) / np.subtract(gpp_array_train.max(),
                                                                                      gpp_array_train.min())

  pr_array_train = np.subtract(pr_array_train, pr_array_train.min()) / np.subtract(pr_array_train.max(),
                                                                                  pr_array_train.min())

  tas_array_train = np.subtract(tas_array_train, tas_array_train.min()) / np.subtract(tas_array_train.max(),
                                                                                      tas_array_train.min())

  mask_train = np.concatenate(mask_train)

  tas_array_test = np.concatenate(tas_array_test)
  print(gpp_array_test)
  pr_array_test = np.concatenate(pr_array_test)
  gpp_array_test = np.concatenate(gpp_array_test)

  tas_array_test = np.subtract(tas_array_test, tas_array_test.min()) / np.subtract(tas_array_test.max(),
                                                                                  tas_array_test.min())

  pr_array_test = np.subtract(pr_array_test, pr_array_test.min()) / np.subtract(pr_array_test.max(), pr_array_test.min())

  gpp_test_min = gpp_array_test.min()
  gpp_test_max = gpp_array_test.max()

  gpp_array_test = np.subtract(gpp_array_test, gpp_array_test.min()) / np.subtract(gpp_array_test.max(),
                                                                                  gpp_array_test.min())

  mask_test = np.concatenate(mask_test)
  train_data = np.stack([tas_array_train,pr_array_train], axis=-1)

  test_data = np.stack([tas_array_test,pr_array_test], axis=-1)

  return train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas


def train_model(train_data,test_data,gpp_array_test,gpp_array_train,dir,model_name,model):
  
  if(model=='unet'):
    model=unet(test_data.shape[1],test_data.shape[2],test_data.shape[3])
  elif(model=='att_unet'):
    model=att_unet(test_data.shape[1],test_data.shape[2],test_data.shape[3])
  elif(model=='att_r2_unet'):
    model=att_r2_unet(test_data.shape[1],test_data.shape[2],test_data.shape[3])
  
  
  
  model.summary()
  model.compile(optimizer=Adam(learning_rate=0.0001),
                loss=tf.keras.losses.MeanAbsoluteError(),
                )

  
  log_dir = dir+"/log1/" + model_name+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  callbacks = [
      ModelCheckpoint(
          # Path where to save the model
          # The two parameters below mean that we will overwrite
          # the current checkpoint if and only if
          # the `val_loss` score has improved.
          # The saved model name will include the current epoch.
          filepath=dir+model_name,
          save_best_only=True,  # Only save a model if `val_loss` has improved.
          monitor="val_loss",
          verbose=1,
      ),
      EarlyStopping(
          # Stop training when `val_loss` is no longer improving
          monitor="val_loss",
          # "no longer improving" being defined as "no better than 1e-2 less"
          min_delta=1e-3,
          # "no longer improving" being further defined as "for at least 2 epochs"
          patience=25,
          verbose=1,
      ),
      TensorBoard(
      log_dir=log_dir,
      histogram_freq=1,  # How often to log histogram visualizations
      embeddings_freq=1,  # How often to log embedding visualizations
      update_freq="epoch",
      )
  ]


  model.fit(
      {"inputs":train_data,},
      {"outputs": gpp_array_train },
      batch_size=32,
      epochs=200,
      callbacks=callbacks,
      validation_split=0.1
  )



def test_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,plot_dir,cubes_tas):
  import matplotlib.cm as mpl_cm
  model = tf.keras.models.load_model(model_dir)

  z = model({"inputs": test_data})
  val = z.numpy()
  val1 = np.reshape(val, (test_data.shape[0], test_data.shape[1], test_data.shape[2]))
  finale_array = val1 * gpp_test_max

  import numpy.ma as ma
  finale_array[finale_array<0]=0
  finale_array=ma.masked_array(finale_array,mask_test)
  brewer_cmap = mpl_cm.get_cmap('brewer_PiYG_11')



  start=0
  input_count=1
  outputcount=1
  for z in cubes_tas:
      # try:
          if (z._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
              temp = z.extract(time_constraint_test)
              temp1=temp.copy()
              temp1.data=finale_array[start:(start+temp.shape[0]),:,:]
              start+=temp.shape[0]
              for x,y in zip(temp.slices_over('time'),temp1.slices_over('time')):
                fig = plt.figure(figsize=(20, 5),dpi=200)
                plt.subplot(131)
                qplt.contourf(x,brewer_cmap.N, cmap=brewer_cmap)
                plt.title('GPP  '+str(x.coord('year').points[0]))
                
                
                #plt.show()
                plt.subplot(132)
                qplt.contourf(y,brewer_cmap.N, cmap=brewer_cmap)
                plt.title('Predicted GPP '+str(x.coord('year').points[0]))
                
                

                plt.subplot(133)
                qplt.contourf(x-y,brewer_cmap.N, cmap=brewer_cmap)
                plt.title('Difference'+str(x.coord('year').points[0]))
                #plt.show()
                plt.savefig(plot_dir+'Plot'+str(input_count) )
                plt.close('all')
                input_count+=1







from pathlib import Path



run=3

time_constraint_train = iris.Constraint(year=lambda cell: 2000 < cell < 2300)
time_constraint_test = iris.Constraint(year=lambda cell: 1960 < cell < 2000)

data_dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/annual/*BCC*.nc'
train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas=getData(data_dir,time_constraint_train,time_constraint_test)
model_dir="/content/drive/MyDrive/models_new/"
model_name='annual_bcc_r2_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_r2_unet')
model_name='annual_bcc_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'unet')
model_name='annual_bcc_att_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_unet')

# model_dir=dir+model_name
# plot_dir='/content/drive/MyDrive/plots/annual_ukesm/run'+str(run)+'/'
# Path(plot_dir).mkdir(parents=True, exist_ok=True)
# test_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,plot_dir,cubes_tas)




dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/jja/*BCC*.nc'
train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas=getData(data_dir,time_constraint_train,time_constraint_test)
model_dir="/content/drive/MyDrive/models_new/"
model_name='jja_bcc_r2_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_r2_unet')
model_name='jja_bcc_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'unet')
model_name='jja_bcc_att_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_unet')

# model_dir=dir+model_name
# plot_dir='/content/drive/MyDrive/plots/jja_ukesm/run'+str(run)+'/'
# Path(plot_dir).mkdir(parents=True, exist_ok=True)
# test_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,plot_dir,cubes_tas)





dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/djf/*BCC*.nc'
train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas=getData(data_dir,time_constraint_train,time_constraint_test)
dir="/content/drive/MyDrive/models_new/"
model_dir="/content/drive/MyDrive/models_new/"
model_name='djf_bcc_r2_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_r2_unet')
model_name='djf_bcc_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'unet')
model_name='djf_bcc_att_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_unet')

# model_dir=dir+model_name

# plot_dir='/content/drive/MyDrive/plots/djf_ukesh/run'+str(run)+'/'
# Path(plot_dir).mkdir(parents=True, exist_ok=True)
# test_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,plot_dir,cubes_tas)


time_constraint_train = iris.Constraint(year=lambda cell: 2000 < cell < 3059)
time_constraint_test = iris.Constraint(year=lambda cell: 1960 < cell < 2000)

data_dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/annual/*UKE*.nc'
train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas=getData(data_dir,time_constraint_train,time_constraint_test)
model_dir="/content/drive/MyDrive/models_new/"
model_name='annual_bcc_r2_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_r2_unet')
model_name='annual_bcc_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'unet')
model_name='annual_bcc_att_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_unet')

# model_dir=dir+model_name
# plot_dir='/content/drive/MyDrive/plots/annual_ukesm/run'+str(run)+'/'
# Path(plot_dir).mkdir(parents=True, exist_ok=True)
# test_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,plot_dir,cubes_tas)




dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/jja/*UKE*.nc'
train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas=getData(data_dir,time_constraint_train,time_constraint_test)
model_dir="/content/drive/MyDrive/models_new/"
model_name='jja_bcc_r2_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_r2_unet')
model_name='jja_bcc_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'unet')
model_name='jja_bcc_att_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_unet')

# model_dir=dir+model_name
# plot_dir='/content/drive/MyDrive/plots/jja_ukesm/run'+str(run)+'/'
# Path(plot_dir).mkdir(parents=True, exist_ok=True)
# test_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,plot_dir,cubes_tas)





dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/djf/*UKE*.nc'
train_data,test_data,gpp_array_test,gpp_array_train,mask_test,cubes_tas=getData(data_dir,time_constraint_train,time_constraint_test)
dir="/content/drive/MyDrive/models_new/"
model_dir="/content/drive/MyDrive/models_new/"
model_name='djf_bcc_r2_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_r2_unet')
model_name='djf_bcc_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'unet')
model_name='djf_bcc_att_unet'
train_model(train_data,test_data,gpp_array_test,gpp_array_train,model_dir,model_name,'att_unet')





Code to test on saved  model and estimate gpp

In [ ]:
import iris
import iris.analysis.cartography
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np
import datetime


import tensorflow as tf
from tensorflow.keras.layers import multiply,add,Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, UpSampling2D, UpSampling3D, Add, BatchNormalization, Input, Activation, Lambda, Cropping2D,Concatenate
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp

import glob

from keras.callbacks import ModelCheckpoint,EarlyStopping,TensorBoard

from tensorflow.keras.optimizers import Adam


list1 = []

run=5
dir = '/content/drive/MyDrive/U-Birmingham-gpp-data/annual/*UKE*.nc'
model_name='annual_ukesm'
model_path='/content/drive/MyDrive/models/'+model_name
# for name in glob.glob(dir):
#     list1.append(name)

from pathlib import Path
plot_dir='/content/drive/MyDrive/plots/'+model_name+'/run'+str(run)+'/'
Path(plot_dir).mkdir(parents=True, exist_ok=True)
cubes_tas = []
for name in glob.glob(dir):
    list1.append(name)

cubes_tas = iris.load(list1)

print(list1)
# equalise_attributes(cubes_tas)
# unify_time_units(cubes_tas)
# names = [cube.metadata.name() for cube in cubes ]
# unique_names = list(OrderedDict.fromkeys(names))


for z in cubes_tas:
    try:

        if (z.coord('season_year')):
            year = z.coord('season_year')
            # Fix the names. Latitude values, units and
            year.var_name = 'year'
            year.long_name = 'year'
    except:
        print("An exception occurred")

# Fix the names. Latitude values, units and


for z in cubes_tas:
    try:
        z.remove_coord("clim_season")
    except:
        print("An exception occurred")

for z in cubes_tas:
    try:
        if (z._names[0] == 'precipitation_flux'):
            z.convert_units('kg m-2 days-1')
        if (z._names[0] == 'air_temperature'):
            z.convert_units('celsius')
        if (z._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
            z.convert_units('kg m-2 year-1')

    except:
        print("An exception occurred")

time_constraint_train = iris.Constraint(year=lambda cell: 2000 < cell < 3049)
time_constraint_test = iris.Constraint(year=lambda cell: 1960 < cell < 2000)

tas_array_train = []
pr_array_train = []
gpp_array_train = []

tas_array_test = []
pr_array_test = []
gpp_array_test = []
mask_train = []
mask_test = []

for z in cubes_tas:
    try:
        if (z._names[0] == 'precipitation_flux'):
            temp = z.extract(time_constraint_train)
            mask_train.append(temp.data._mask)
            pr_array_train.append(temp.data.filled(0))
            temp = z.extract(time_constraint_test)
            mask_test.append(temp.data._mask)
            pr_array_test.append(temp.data.filled(0))
        if (z._names[0] == 'air_temperature'):
            temp = z.extract(time_constraint_train)
            tas_array_train.append(temp.data.filled(0))
            temp = z.extract(time_constraint_test)
            tas_array_test.append(temp.data.filled(0))
        if (z._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
            temp = z.extract(time_constraint_train)
            gpp_array_train.append(temp.data.filled(0))
            temp = z.extract(time_constraint_test)
            gpp_array_test.append(temp.data.filled(0))

    except:
        print("An exception occurred")

tas_array_train = np.concatenate(tas_array_train)
pr_array_train = np.concatenate(pr_array_train)
gpp_array_train = np.concatenate(gpp_array_train)

gpp_array_train = np.subtract(gpp_array_train, gpp_array_train.min()) / np.subtract(gpp_array_train.max(),
                                                                                    gpp_array_train.min())

pr_array_train = np.subtract(pr_array_train, pr_array_train.min()) / np.subtract(pr_array_train.max(),
                                                                                 pr_array_train.min())

tas_array_train = np.subtract(tas_array_train, tas_array_train.min()) / np.subtract(tas_array_train.max(),
                                                                                    tas_array_train.min())

mask_train = np.concatenate(mask_train)

tas_array_test = np.concatenate(tas_array_test)
pr_array_test = np.concatenate(pr_array_test)
gpp_array_test = np.concatenate(gpp_array_test)

tas_array_test = np.subtract(tas_array_test, tas_array_test.min()) / np.subtract(tas_array_test.max(),
                                                                                 tas_array_test.min())

pr_array_test = np.subtract(pr_array_test, pr_array_test.min()) / np.subtract(pr_array_test.max(), pr_array_test.min())

gpp_test_min = gpp_array_test.min()
gpp_test_max = gpp_array_test.max()

gpp_array_test = np.subtract(gpp_array_test, gpp_array_test.min()) / np.subtract(gpp_array_test.max(),
                                                                                 gpp_array_test.min())

mask_test = np.concatenate(mask_test)
train_data = np.stack([tas_array_train,pr_array_train], axis=-1)

test_data = np.stack([tas_array_test,pr_array_test], axis=-1)

model = tf.keras.models.load_model(model_path)
# model.summary





z = model({"inputs": test_data})
val = z.numpy()
val1 = np.reshape(val, (test_data.shape[0], test_data.shape[1], test_data.shape[2]))
finale_array = val1 * gpp_test_max

import numpy.ma as ma
# finale_array[finale_array<0]=0
finale_array=ma.masked_array(finale_array,mask_test)
import matplotlib.cm as mpl_cm

brewer_cmap = mpl_cm.get_cmap('brewer_PiYG_11')



start=0
input_count=1
outputcount=1
for z in cubes_tas:
    # try:
        if (z._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
            temp = z.extract(time_constraint_test)
            temp1=temp.copy()
            temp1.data=finale_array[start:(start+temp.shape[0]),:,:]
            start+=temp.shape[0]
            for x,y in zip(temp.slices_over('time'),temp1.slices_over('time')):
                fig = plt.figure(figsize=(20, 5),dpi=200)
                plt.subplot(131)
                qplt.contourf(x,brewer_cmap.N, cmap=brewer_cmap)
                plt.title('GPP  '+str(x.coord('year').points[0]))
                
                
                #plt.show()
                plt.subplot(132)
                qplt.contourf(y,brewer_cmap.N, cmap=brewer_cmap)
                plt.title('Predicted GPP '+str(x.coord('year').points[0]))
                
                

                plt.subplot(133)
                qplt.contourf(x-y,brewer_cmap.N, cmap=brewer_cmap)
                plt.title('Difference'+str(x.coord('year').points[0]))
                # plt.show()
                plt.savefig(plot_dir +
                            'input'+str(input_count) )
                plt.close('all')
                input_count+=1

Model plots using tensorboard

In [ ]:
# !pip install tensorboard
%load_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir /content/drive/MyDrive/models_new/log